In [1]:
import json
import pandas as pd
import re
import ast

# PlumX Metrics

In [18]:
f = open('plumx_metrics_data.json')
data_json = json.load(f)

In [33]:
data_flat = pd.DataFrame()
for item in data_json:
    item_flat =  pd.json_normalize(item)
    data_flat = pd.concat([data_flat, item_flat], axis=0, join='outer')

In [35]:
data_flat.to_csv("plumx_metrics_cleaned.csv", index=False)

# Altmetrics

In [4]:
f = open('altmetric_data.json')
altmetrics_corpus = f.read()
altmetrics_corpus = altmetrics_corpus.replace('\\', '')
altmetrics_corpus = altmetrics_corpus.replace(',', ', ')
altmetrics_corpus = altmetrics_corpus.replace(':', ': ')
altmetrics_corpus = altmetrics_corpus[1:-1]

In [26]:
index = 0
item_start_index = None
item_end_index = None
total = len(altmetrics_corpus)
stack = 0
item_indices = list()
while(index < total):
    if(altmetrics_corpus[index]=='{'):
        if(stack == 0): # opening new item 
            item_start_index = index
            stack+=1
            index+=1 
        else: # nested data within item 
            stack+=1
            index+=1
    elif(altmetrics_corpus[index]=='}'):
        if(stack == 1): # closing item
            item_end_index = index+1
            item_indices.append((item_start_index, item_end_index))
            stack-=1
            index+=1
        else: # closing nested data within item
            stack-=1
            index+=1
    else:
        index+=1   

In [28]:
len(item_indices) # should be 7963

7963

In [47]:
data_flat = pd.DataFrame()
for item in item_indices:
    text = altmetrics_corpus[item[0]:item[1]]
    try:
        item_dict = json.loads(text)
        item_flat =  pd.json_normalize(item_dict)
        data_flat = pd.concat([data_flat, item_flat], axis=0, join='outer')
    except:
        print("Unable to parse start index ", item[0], " end index ", item[1], " .")

Unable to parse start index  121759  end index  126277  .
Unable to parse start index  184992  end index  188193  .
Unable to parse start index  294550  end index  297754  .
Unable to parse start index  324939  end index  329156  .
Unable to parse start index  745882  end index  748817  .
Unable to parse start index  1030354  end index  1034536  .
Unable to parse start index  1127726  end index  1132586  .
Unable to parse start index  1187176  end index  1191768  .
Unable to parse start index  1313121  end index  1316683  .
Unable to parse start index  1718613  end index  1722656  .
Unable to parse start index  1759977  end index  1763670  .
Unable to parse start index  1828477  end index  1832003  .
Unable to parse start index  1843672  end index  1847069  .
Unable to parse start index  2084118  end index  2092949  .
Unable to parse start index  2768300  end index  2773647  .
Unable to parse start index  3089648  end index  3092618  .
Unable to parse start index  3311248  end index  3

We will ignore the parsing errors for now.  They appear to be caused by the use of quotation marks in the abstract and 1.4% of the samples are affected.  This will be addressed in a later update.

In [48]:
data_flat

,title,doi,isbns,altmetric_jid,issns,journal,cohorts,authors,type,handles,...,cited_by_rdts_count,cited_by_peer_review_sites_count,editors,ordinal_number,hlom_id,hollis_id,authors_or_editors,attribution,cited_by_linkedin_count,cited_by_weibo_count
0,Isolation and Characterization of the Saturate...,10.1021/ef990207h,[],4f6fa56d3cf058f610004b77,"[0887-0624, 1520-5029]",Energy & Fuels,[],"[Walter E. Rudzinski, Tejraj M. Aminabhavi, St...",article,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,A Review on Extraction and Identification of C...,10.1021/ef990177i,[],4f6fa56d3cf058f610004b77,"[0887-0624, 1520-5029]",Energy & Fuels,[],"[Walter E. Rudzinski, Tejraj M. Aminabhavi]",article,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Antimicrobial Residue Detection in Chicken Yol...,10.1021/jf000140s,[],4f6fa6053cf058f610006625,"[0021-8561, 1520-5118]",Journal of Agricultural & Food Chemistry,[],"[Jackson L. McReynolds, Denise Y. Caldwell, Au...",article,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Effect of acute postexercise ethanol intoxicat...,10.1152/jappl.2000.88.1.165,NaN,4f6fa61f3cf058f610007b71,"[8750-7587, 1522-1601]",Journal of Applied Physiology,NaN,"[L. Perry Koziris, William J. Kraemer, Scott E...",article,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Electrical and magnetic properties of La0.5Sr0...,10.1063/1.126208,[],4f6fa4e83cf058f6100022e1,"[0003-6951, 1077-3118]",Applied Physics Letters,[],"[G. P. Luo, Y. S. Wang, S. Y. Chen, A. K. Heil...",article,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Rights amidst commercial gentrification: a co...,10.1080/02697459.2024.2351759,[],4f6fa4d93cf058f6100014ac,"[0269-7459, 1360-0583]",Planning Practice and Research,NaN,"[Janet Muu00f1iz, Ivis Garcu00eda, Erualdo Gon...",article,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Visibility of inflicted bruises by alternate l...,10.1111/1556-4029.15481,[],4f6fa4f53cf058f610002db2,"[0022-1198, 1556-4029]",Journal of Forensic Sciences (Blackwell Publis...,NaN,"[Nancy R. Downing, Katherine N. Scafide, Zahra...",article,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Biotene: Earth-Abundant 2D Material as Sustai...,10.1021/acsami.3c15664,[],4f6fa6073cf058f61000680f,"[1944-8244, 1944-8252]",ACS Applied Materials & Interfaces,NaN,"[Atin Pramanik, Preeti Lata Mahapatra, Raphael...",article,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Association of Mild Iodine Insufficiency durin...,10.1055/s-0043-1778037,NaN,4f6fa4e83cf058f610002207,"[0735-1631, 1098-8785]",American Journal of Perinatology,NaN,"[Brian M. Casey, Lisa Mele, Alan M. Peaceman, ...",article,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
data_flat.to_csv("altmetrics_cleaned.csv", index=False)